In [1]:
import pokerSim_Cards as pSC
import Poker as poker
import random

class pokerSim_Player:
    def __init__(self,stack):
        self.stack = stack
        self.hand = []
        self.handIndex = []
        self.bet = 0
    def toIndex(self):
        if(self.rank == 0):
            r = 12
        else:
            r = rank-1
        return r+self.suit*13
    def handToIndex(self):
        HAND = self.hand
        results = []
        for card in HAND:
            if card.rank == 0:
                r = 12
            else:
                r = card.rank - 1
            results += [r+card.suit*13]
        return results
        
class pokerSim_Game:
    def __init__(self,playerAStack,playerBStack):
        self.playerA = pokerSim_Player(playerAStack)
        self.playerB = pokerSim_Player(playerBStack)
        self.board = pokerSim_Board()
        self.gameCounter = 0
        
class pokerSim_Board:
    def __init__(self):
        self.deck = pSC.pokerSim_Deck()
        self.board = []
        for i in range(5):
            self.board.append(self.deck.deck[i])
        self.pot = 0
        self.highestBet = 0
        
    def boardToIndex(self):
        BOARD = self.board
        results = []
        for card in BOARD:
            if card.rank == 0:
                r = 12
            else:
                r = card.rank - 1
            results += [r+card.suit*13]
        return results

In [2]:
GAME_STATE_GAME_INIT = 1
GAME_STATE_PREFLOP = 2
GAME_STATE_FLOP = 3
GAME_STATE_TURN = 4
GAME_STATE_RIVER = 5
GAME_STATE_SHOWDOWN = 6

GAME_SMALL_BLIND = 10
GAME_BIG_BLIND = GAME_SMALL_BLIND*2

GAME_END_PLAYER_A_WINS = 1
GAME_END_PLAYER_B_WINS = 2

GAME_PLAYER_A = 1
GAME_PLAYER_B = 2


def allInit(playerAStack=500,playerBStack=500):
    return pokerSim_Game(playerAStack,playerBStack) 

def dealCards(game):
    game.gameCounter += 1
    print("\ngame "+str(game.gameCounter)+" :\n")
    game.board = pokerSim_Board()
    game.playerA.hand = []
    game.playerB.hand = []
    game.playerA.hand.append(game.board.deck.deck[5]) 
    game.playerA.hand.append(game.board.deck.deck[6])
    game.playerB.hand.append(game.board.deck.deck[7]) 
    game.playerB.hand.append(game.board.deck.deck[8])
    print("Cards are dealt: ")
    print("Board:")
    for i in range(5):
        print(game.board.board[i])
    print("\nplayer A hand: ")
    for i in range(2):
        print(game.playerA.hand[i])
    print("\nplayer B hand: ")
    for i in range(2):
        print(game.playerB.hand[i])
    return game
        
def gameInit(game,state):
    game = dealCards(game)
    if(game.playerA.stack == 0):
        return (game,state,GAME_END_PLAYER_B_WINS)
    elif(game.playerB.stack == 0):
        return (game,state,GAME_END_PLAYER_A_WINS)
    else:
        state+=1
        return (game,state,0)
def printGameInfo(game):
    print("playerA stack: "+str(game.playerA.stack))
    print("playerA bet: "+str(game.playerA.bet))
    print("playerB stack: "+str(game.playerB.stack))
    print("playerB bet: "+str(game.playerB.bet))
    print("board pot: "+str(game.board.pot))
    print("board highestBet: "+str(game.board.highestBet)+"\n")
    

def gameBet(game,betChips,player):
    if(player == GAME_PLAYER_A):#player A
        game.playerA.stack -= betChips
        game.board.pot += betChips
        game.playerA.bet += betChips
        game.board.highestBet = max(game.board.highestBet,game.playerA.bet)
    else: #player B
        game.playerB.stack -= betChips
        game.board.pot += betChips
        game.playerB.bet += betChips
        game.board.highestBet = max(game.board.highestBet,game.playerB.bet)
    return game

GAME_MOVE_DEFAULT = 0
GAME_MOVE_CALL = 1
GAME_MOVE_FOLD = 2
GAME_MOVE_CHECK = 3
GAME_MOVE_RAISE = 4
GAME_MOVE_CALL_PREFLOP = 5

def smallBlindBigBlindBet(game):
    if(game.playerA.stack <= GAME_SMALL_BLIND): #all in 
        gameBet(game,game.playerA.stack,GAME_PLAYER_A)
        print("playerA doesn't have enough chips more than small blind, choose to all in instead.\n")
    else:
        game = gameBet(game,GAME_SMALL_BLIND,GAME_PLAYER_A)
        print("playerA place "+ str(GAME_SMALL_BLIND)+" in the pot.\n")
    if(game.playerB.stack <= GAME_BIG_BLIND):
        gameBet(game,game.playerB.stack,GAME_PLAYER_B)
        print("playerB doesn't have enough chips more than big blind, choose to all in instead.\n")
    else:
        game = gameBet(game,GAME_BIG_BLIND,GAME_PLAYER_B)
        print("playerB place "+ str(GAME_BIG_BLIND)+" in the pot.\n")
    printGameInfo(game)
    return game

def playFold(game,state,player):
    if(player == GAME_PLAYER_A): #player A
        print("player A chooses to fold, this game is over")
        game.playerA.bet = 0
        game.playerB.stack += game.board.pot
        game.playerB.bet = 0
        state = GAME_STATE_GAME_INIT
    else: #player B
        print("player B chooses to fold, this game is over")
        game.playerB.bet = 0
        game.playerA.stack += game.board.pot
        game.playerA.bet = 0
        state = GAME_STATE_GAME_INIT
    return game,state,1,0


def playCall(game,state,player,opponentMove):
    if(player == GAME_PLAYER_A): #player A
        print("player A chooses to call:")
        bet = game.board.highestBet - game.playerA.bet
        if(game.playerA.stack<bet):
            gameBet(game,game.playerA.stack,GAME_PLAYER_A)
            print("playerA doesn't have enough chips more than small blind, choose to all in instead.\n")
        else:
            game = gameBet(game,bet,player)
    else:
        print("player B chooses to call:")
        bet = game.board.highestBet - game.playerB.bet
        if(game.playerB.stack<bet):
            gameBet(game,game.playerB.stack,GAME_PLAYER_B)
            print("playerB doesn't have enough chips more than small blind, choose to all in instead.\n")
        else:
            game = gameBet(game,bet,player)
    printGameInfo(game)
    if(state == GAME_STATE_PREFLOP and player == GAME_PLAYER_A and opponentMove == GAME_MOVE_DEFAULT):
        return game,state,0,0
    else:
        state += 1
        return game,state,1,0
def playCheck(game,state,player,opponentMove):
    if(player == GAME_PLAYER_A):
        print("player A chooses to check:")
        if(opponentMove==GAME_MOVE_CHECK or opponentMove == GAME_MOVE_CALL or opponentMove == GAME_MOVE_CALL_PREFLOP):
            state += 1
    else:
        print("player B chooses to check:")
        if(opponentMove==GAME_MOVE_CHECK or opponentMove == GAME_MOVE_CALL or opponentMove == GAME_MOVE_CALL_PREFLOP):
            state += 1   
    printGameInfo(game)  
    return game,state,1,0
def playRaise(game,state,player,playerBet):
    if(player == GAME_PLAYER_A): #player A
        if(game.playerB.stack == 0):
            print("player B has been all in, can't raise anymore.")
            state+=1
            return(game,state,1,0)
        print("player A chooses to raise:")
        game = gameBet(game,playerBet,GAME_PLAYER_A)
    else: #player B
        if(game.playerA.stack == 0):
            print("player A has been all in, can't raise anymore.")
            state+=1
            return(game,state,1,0)
        print("player B chooses to raise:")
        game = gameBet(game,playerBet,GAME_PLAYER_B)   
    printGameInfo(game)
    return (game,state,0,0)
def chooseMovePlayer(game,state,player,opponentMove):
    if(player == GAME_PLAYER_A): #player A
        if(opponentMove == GAME_MOVE_DEFAULT and state == GAME_STATE_PREFLOP): #special call in preflop
            #print("Bonnie")
            #print(game.playerA.handToIndex())
            #print("anything above?")
            #print(game.playerA.hand[0])
            #print(game.playerA.hand[1])
            #print(game.playerB.hand[0])
            #print(game.playerB.hand[1])

            playerAMove = poker.Move(hand=game.playerA.handToIndex(), board=game.board.boardToIndex(), pot=game.board.pot, BB = 20, 
                                     stacks = [game.playerA.stack, game.playerB.stack], turn = 0, bet = 0, state = state - 2, alpha = 0.7)
            if playerAMove is None:
                    playerAMove = (2,0)
            playerAM = playerAMove[0]
            playerAB = playerAMove[1]
            playerAB = playerAMove[1]-game.playerA.bet
            if(playerAM == GAME_MOVE_RAISE):
                playerABet = playerAB
            if(playerAM == GAME_MOVE_CALL):
                if(game.playerA.bet >= game.board.highestBet):
                    if(game.playerB.stack == 0):
                        game,state,roundIsOver,gameIsOver = playCheck(game,state,GAME_PLAYER_A,opponentMove)
                        return game,state,roundIsOver,gameIsOver,GAME_MOVE_CHECK
                    else:
                        print("playerA can't call, chooses to fold instead.")
                        game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                        return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                else:
                    game,state,roundIsOver,gameIsOver = playCall(game,state,GAME_PLAYER_A,opponentMove)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_CALL_PREFLOP
            elif(playerAM == GAME_MOVE_FOLD):
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerAM == GAME_MOVE_CHECK):
                print("playerA can't check, chooses to fold instead.")
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerAM == GAME_MOVE_RAISE):
                if(playerABet>game.playerA.stack or playerABet<=game.board.highestBet - game.playerA.bet): #illegal move
                    playerABet = min(game.board.highestBet - game.playerA.bet+1,game.playerA.stack)
                    #print("playerA can't raise since the bet number is higher than his stack or lower than the bet gap, chooses to fold instead")
                    #game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                    #return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
                else:
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
        elif(opponentMove == GAME_MOVE_RAISE):
            playerAMove = poker.Move(hand=game.playerA.handToIndex(), board=game.board.boardToIndex(), pot=game.board.pot, BB = 20, 
                                     stacks = [game.playerA.stack, game.playerB.stack], turn = 0, bet = game.playerB.bet, state = state - 2, alpha = 0.7)
            if playerAMove is None:
                    playerAMove = (2,0)            
            playerAM = playerAMove[0]
            playerAB = playerAMove[1]
            playerAB = playerAMove[1]-game.playerA.bet
            if(playerAM == GAME_MOVE_RAISE):
                playerABet = playerAB
            if(playerAM == GAME_MOVE_CALL):
                if(game.playerA.bet >= game.board.highestBet):
                    print("playerA can't call, chooses to fold instead.")
                    game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                else:
                    game,state,roundIsOver,gameIsOver = playCall(game,state,GAME_PLAYER_A,opponentMove)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_CALL
            elif(playerAM == GAME_MOVE_FOLD):
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerAM == GAME_MOVE_CHECK):
                print("playerA can't check, chooses to fold instead.")
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerAM == GAME_MOVE_RAISE):
                if(playerABet>game.playerA.stack or playerABet<=game.board.highestBet - game.playerA.bet): #illegal move
                    playerABet = min(game.board.highestBet - game.playerA.bet+1,game.playerA.stack)
                    #print("playerA can't raise since the bet number is higher than his stack or lower than the bet gap, chooses to fold instead")
                    #game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                    #return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
                else:
                    playerABet = playerAB
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
        elif(opponentMove == GAME_MOVE_FOLD):
            return game,state,1,0,GAME_MOVE_FOLD
        elif(opponentMove == GAME_MOVE_CHECK):
            playerAMove = poker.Move(hand=game.playerA.handToIndex(), board=game.board.boardToIndex(), pot=game.board.pot, BB = 20, stacks = [game.playerA.stack, game.playerB.stack], turn = 0, bet = 0, state = state - 2, alpha = 0.7)
            if playerAMove is None:
                playerAMove = (2,0)   
            playerAM = playerAMove[0]
            playerAB = playerAMove[1]
            playerAB = playerAMove[1]-game.playerA.bet
            if(playerAM == GAME_MOVE_RAISE):
                playerABet = playerAB
            if(playerAM == GAME_MOVE_CALL):
                print("playerA can't call, chooses to fold instead.")
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerAM == GAME_MOVE_FOLD):
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerAM == GAME_MOVE_CHECK):
                game,state,roundIsOver,gameIsOver = playCheck(game,state,GAME_PLAYER_A,opponentMove)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_CHECK
            elif(playerAM == GAME_MOVE_RAISE):
                if(playerABet>game.playerA.stack or playerABet<=game.board.highestBet - game.playerA.bet): #illegal move
                    playerABet = min(game.board.highestBet - game.playerA.bet+1,game.playerA.stack)
                    #print("playerA can't raise since the bet number is higher than his stack or lower than the bet gap, chooses to fold instead")
                    #game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_A)
                    #return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
                else:
                    playerBet = playerAB
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
        elif(opponentMove == GAME_MOVE_CALL):
            return game,state,1,0,GAME_MOVE_CHECK
    else: #player B
        if(state == GAME_STATE_PREFLOP and opponentMove == GAME_MOVE_CALL_PREFLOP):
            playerBMove = poker.Move(hand=game.playerB.handToIndex(), board=game.board.boardToIndex(), pot=game.board.pot, BB = 20, stacks = [game.playerA.stack, game.playerB.stack], turn = 1, bet = 0, state = 0, alpha = 0.4)
            if playerBMove is None:
                playerBMove = (2,0) 
            playerBM = playerBMove[0]
            playerBB = playerBMove[1]
            playerBB = playerBMove[1]-game.playerB.bet
            if(playerBM == GAME_MOVE_RAISE):
                playerBBet = playerBB
            if(playerBM == GAME_MOVE_CALL):
                print("playerB can't call, chooses to check instead")
                game,state,roundIsOver,gameIsOver = playCheck(game,state,GAME_PLAYER_B,opponentMove)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_CHECK
            elif(playerBM == GAME_MOVE_FOLD):
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerBM == GAME_MOVE_CHECK):
                game,state,roundIsOver,gameIsOver = playCheck(game,state,GAME_PLAYER_B,opponentMove)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_CHECK
            elif(playerBM == GAME_MOVE_RAISE): #raise
                playerBBet = playerBB
                if(playerBBet>game.playerB.stack or playerBBet<=game.board.highestBet - game.playerB.bet): #illegal move
                    #print("playerB can't raise since the bet number is higher than his stack or lower than the bet gap, chooses to fold instead")
                    playerBBet = min(game.board.highestBet - game.playerB.bet+1,game.playerB.stack)
                    game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                else:
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_B,playerBBet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
        elif(opponentMove == GAME_MOVE_CHECK):
            game,state,roundIsOver,gameIsOver = playCheck(game,state,GAME_PLAYER_B,opponentMove)
            return game,state,roundIsOver,gameIsOver,GAME_MOVE_CHECK
        elif(opponentMove == GAME_MOVE_CALL):
            return game,state,1,0,GAME_MOVE_CHECK
        elif(opponentMove == GAME_MOVE_FOLD):
            return game,state,1,0,GAME_MOVE_FOLD
        elif(opponentMove == GAME_MOVE_RAISE):
            playerBMove = (2,0)
            playerBMove = poker.Move(hand=game.playerB.handToIndex(), board=game.board.boardToIndex(), pot=game.board.pot, BB = 20,stacks = [game.playerA.stack, game.playerB.stack], turn = 1, bet = game.playerA.bet, state = state - 2, alpha = 0.4)
            if playerBMove is None:
                playerBMove = (2,0) 
            playerBM = playerBMove[0]
            playerBB = playerBMove[1]
            playerBB = playerBMove[1]-game.playerB.bet
            if(playerBM == GAME_MOVE_RAISE):
                playerBBet = playerBB
            if(playerBM == GAME_MOVE_CALL):
                game,state,roundIsOver,gameIsOver = playCall(game,state,GAME_PLAYER_B,opponentMove)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_CALL
            elif(playerBM == GAME_MOVE_FOLD):
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerBM== GAME_MOVE_CHECK):
                print("playerB can't check, chooses to fold instead.")
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerBM == GAME_MOVE_RAISE): #raise
                playerBBet = playerBB
                if(playerBBet>game.playerB.stack or playerBBet<=game.board.highestBet - game.playerB.bet): #illegal move
                    playerBBet = min(game.board.highestBet - game.playerB.bet+1,game.playerB.stack)
                    game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                    #game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_A,playerABet)
                    #return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE                    print("playerB can't raise since the bet number is higher than his stack or lower than the bet gap, chooses to fold instead")
                    #game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                    #return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                else:
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_B,playerBBet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
        elif(opponentMove == GAME_MOVE_DEFAULT):
            playerBMove = poker.Move(hand=game.playerB.handToIndex(), board=game.board.boardToIndex(), pot=game.board.pot, BB = 20, stacks = [game.playerA.stack, game.playerB.stack], turn = 1, bet = 0, state = state - 2, alpha = 0.4)
            if playerBMove is None:
                playerBMove = (2,0) 
            playerBM = playerBMove[0]
            playerBB = playerBMove[1]
            playerBB = playerBMove[1]-game.playerB.bet
            if(playerBM == GAME_MOVE_RAISE):
                playerBBet = playerBB
            if(playerBM == GAME_MOVE_CALL):
                print("playerB can't call, chooses to fold instead.")
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerBM == GAME_MOVE_FOLD):
                game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
            elif(playerBM == GAME_MOVE_CHECK):
                game,state,roundIsOver,gameIsOver = playCheck(game,state,GAME_PLAYER_B,opponentMove)
                return game,state,roundIsOver,gameIsOver,GAME_MOVE_CHECK
            elif(playerBM == GAME_MOVE_RAISE): #raise
                playerBBet = playerBB
                if(playerBBet>game.playerB.stack or playerBBet<=game.board.highestBet - game.playerB.bet): #illegal move
                    playerBBet = min(game.board.highestBet - game.playerB.bet+1,game.playerB.stack)
                    game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                    #print("playerB can't raise since the bet number is higher than his stack or lower than the bet gap, chooses to fold instead")
                    #game,state,roundIsOver,gameIsOver = playFold(game,state,GAME_PLAYER_B)
                    #return game,state,roundIsOver,gameIsOver,GAME_MOVE_FOLD
                else:
                    game,state,roundIsOver,gameIsOver = playRaise(game,state,GAME_PLAYER_B,playerBBet)
                    return game,state,roundIsOver,gameIsOver,GAME_MOVE_RAISE
    return        
def preflop(game,state):
    print("Round Preflop:\n")
    game = smallBlindBigBlindBet(game)
    roundIsOver = 0
    gameIsOver = 0
    playerAMove = GAME_MOVE_DEFAULT
    playerBMove = GAME_MOVE_DEFAULT
    while(roundIsOver!=1):
        game,state,roundIsOver,gameIsOver,playerAMove = chooseMovePlayer(game,state,GAME_PLAYER_A,playerBMove)
        game,state,roundIsOver,gameIsOver,playerBMove = chooseMovePlayer(game,state,GAME_PLAYER_B,playerAMove)
    return (game,state,gameIsOver)

def flop(game,state):
    print("Round Flop:\n")
    roundIsOver = 0
    gameIsOver = 0
    playerAMove = GAME_MOVE_DEFAULT
    playerBMove = GAME_MOVE_DEFAULT
    while(roundIsOver!=1):
        game,state,roundIsOver,gameIsOver,playerBMove = chooseMovePlayer(game,state,GAME_PLAYER_B,playerAMove)
        game,state,roundIsOver,gameIsOver,playerAMove = chooseMovePlayer(game,state,GAME_PLAYER_A,playerBMove)
    return (game,state,gameIsOver)

def turn(game,state):
    print("Round Turn:\n")
    roundIsOver = 0
    gameIsOver = 0
    playerAMove = GAME_MOVE_DEFAULT
    playerBMove = GAME_MOVE_DEFAULT
    while(roundIsOver!=1):
        game,state,roundIsOver,gameIsOver,playerBMove = chooseMovePlayer(game,state,GAME_PLAYER_B,playerAMove)
        game,state,roundIsOver,gameIsOver,playerAMove = chooseMovePlayer(game,state,GAME_PLAYER_A,playerBMove)
    return (game,state,gameIsOver)

def river(game,state):
    print("Round River:\n")
    roundIsOver = 0
    gameIsOver = 0
    playerAMove = GAME_MOVE_DEFAULT
    playerBMove = GAME_MOVE_DEFAULT
    while(roundIsOver!=1):
        game,state,roundIsOver,gameIsOver,playerBMove = chooseMovePlayer(game,state,GAME_PLAYER_B,playerAMove)
        game,state,roundIsOver,gameIsOver,playerAMove = chooseMovePlayer(game,state,GAME_PLAYER_A,playerBMove)
    return (game,state,gameIsOver)

def showdown(game,state):
    print("Showdown:\n")
    tempA = pSC.getAllPossibilities(game.playerA.hand,game.board.board)
    tempB = pSC.getAllPossibilities(game.playerB.hand,game.board.board)
    BestA = pSC.findBest(tempA)
    BestB = pSC.findBest(tempB)
    result = pSC.compareTwoHands(BestA,BestB)
    if(result == GAME_PLAYER_A):
        print("player A has a better hand!")
        game.playerA.stack+=game.board.pot
        game.playerA.bet = 0
        game.playerB.bet = 0
    elif(result == GAME_PLAYER_B):
        print("player B has a better hand!")
        game.playerB.stack+=game.board.pot
        game.playerA.bet = 0
        game.playerB.bet = 0
    else: #A Draw
        print("It's a draw")
        game.playerA.bet = 0
        game.playerB.bet = 0
        game.playerA.stack+=board.pot//2
        game.playerB.stack+=board.pot//2
    state = GAME_STATE_GAME_INIT
    return (game,state,0)

def gameFlow(state,game):
    gameIsOver = 0
    while(gameIsOver==0):
        if(state == GAME_STATE_GAME_INIT):
            game,state,gameIsOver = gameInit(game,state)
        elif(state == GAME_STATE_PREFLOP):
            game,state,gameIsOver = preflop(game,state)
        elif(state == GAME_STATE_FLOP):
            game,state,gameIsOver = flop(game,state)
        elif(state == GAME_STATE_TURN):
            game,state,gameIsOver = turn(game,state)
        elif(state == GAME_STATE_RIVER):
            game,state,gameIsOver = river(game,state)
        else:
            game,state,gameIsOver = showdown(game,state)
    print("the winner is player:"+str(gameIsOver))
            #turn()
    #elif(state == GAME_STATE_RIVER):
        #river()
    #else:
        #showdown()

In [3]:
game = allInit(100,100) #it is (500,500) in default, but it will be a long game for both players, here you can use (100,100) to have a quick test whether the code works
gameFlow(GAME_STATE_GAME_INIT,game)
#In this example, player A is a normal player while playerB is a very passive player.


game 1 :

Cards are dealt: 
Board:
FOUR of HEARTS
ACE of SPADES
ACE of CLUBS
FOUR of DIAMONDS
QUEEN of CLUBS

player A hand: 
ACE of DIAMONDS
TWO of DIAMONDS

player B hand: 
QUEEN of SPADES
THREE of HEARTS
Round Preflop:

playerA place 10 in the pot.

playerB place 20 in the pot.

playerA stack: 90
playerA bet: 10
playerB stack: 80
playerB bet: 20
board pot: 30
board highestBet: 20

player A chooses to raise:
playerA stack: 10
playerA bet: 90
playerB stack: 80
playerB bet: 20
board pot: 110
board highestBet: 90

player B chooses to call:
playerA stack: 10
playerA bet: 90
playerB stack: 10
playerB bet: 90
board pot: 180
board highestBet: 90

Round Flop:

player B chooses to fold, this game is over

game 2 :

Cards are dealt: 
Board:
TEN of HEARTS
TEN of SPADES
TEN of CLUBS
SIX of HEARTS
JACK of CLUBS

player A hand: 
KING of CLUBS
SIX of CLUBS

player B hand: 
KING of HEARTS
KING of DIAMONDS
Round Preflop:

playerA place 10 in the pot.

playerB doesn't have enough chips more than big 

If the code crashes in the end, we determine the winner by who has more chips in the end